In [1]:
from peft import PeftModel, PeftConfig
from transformers import AutoTokenizer, AutoModelForCausalLM

/home/ysx/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
peft_model_id = "/home/ysx/output_dir/checkpoint-2000/pt_lora_model"
config = PeftConfig.from_pretrained(peft_model_id)
inference_model = AutoModelForCausalLM.from_pretrained(
    "/home/ysx/models/chinese-alpaca-2-7b",
    load_in_8bit=True, 
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained("/home/ysx/models/chinese-alpaca-2-7b")


Loading checkpoint shards: 100%|██████████| 2/2 [00:22<00:00, 11.33s/it]


In [4]:
model = PeftModel.from_pretrained(inference_model, peft_model_id, device_map="auto")

In [5]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(55296, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear8bitLt(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear8bitLt(
                in_features=4096, out_features=4096, bias=

In [20]:
prompt = "岐伯曰：女子七岁肾气盛"

In [21]:
inputs = tokenizer(prompt, return_tensors="pt")

In [22]:
inputs

{'input_ids': tensor([[    1, 29871, 47510, 33178, 35344, 30383, 33164, 31425, 32133, 34825,
         32089, 32961]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [23]:
generate_ids = model.generate(
    input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=100
)

In [24]:
tokenizer.batch_decode(generate_ids.detach().cpu().numpy(), skip_special_tokens=True)[0]

'岐伯曰：女子七岁肾气盛，齿发才露。 二岁肝气盛，齿发而爱怒。\n三岁心气盛，舌齿发而爱言。四岁脾气盛，体解而能言。五岁肺气盛，鼻毛才生。六岁肾气衰，齿发而唇重。七岁肝气衰，鼻毛才生，齿发而唇重也。'